# 1. Recap: Understanding Traditional Merkle Trees
## What is a Merkle Tree?
A Merkle Tree is a data structure used to verify the integrity of data. It's a type of binary tree where each leaf node is a hash of a data block, and each non-leaf node is a hash of its child nodes. This structure allows for efficient and secure verification of data integrity.
## Why Use Merkle Trees?
Merkle Trees are widely used in blockchain technology, distributed systems, and version control systems. They allow you to verify that a piece of data belongs to a larger dataset without needing to access the entire dataset.
## How Does a Merkle Tree Work?
- Leaves: Each leaf node contains a hash of a data block.
- Nodes: Each non-leaf node contains a hash of the concatenation of its child nodes.
- Root: The root node is a single hash that represents the entire dataset.

Example:
Imagine you have four data blocks: A, B, C, and D. The Merkle Tree would look like this:
```
        Root
       /    \
   Hash0    Hash1
   /  \      /  \
 A    B    C    D
```
- Hash0 = hash(A|B)
- Hash1 = hash(C|D)
- Root = hash(Hash0|Hash1)

1. Build the merkle tree -> compute Hash0, Hash1, Root
2. Prove leaf node B is in the tree -> compute the merkle proof [A, Hash1]
3. Verify the merkle proof, ensuring the node B is in the tree. Check hash(hash(A|B) | Hash1) == Root


In [12]:
import hashlib


def hash(data):
    return hashlib.sha256(data.encode()).hexdigest()


class MerkleTree:
    def __init__(self, leaves):
        # fixed-size hash outputs as leaves
        self.leaves = [hash(leaf) for leaf in leaves]
        self.tree = [self.leaves]
        self.build_tree(self.leaves)

    def build_tree(self, current_level):
        if len(current_level) == 1:
            return
        next_level = []
        for i in range(0, len(current_level), 2):
            left = current_level[i]
            right = current_level[i + 1] if i + 1 < len(current_level) else left
            combined = left + right
            next_level.append(hash(combined))
        self.tree.append(next_level)
        self.build_tree(next_level)
    
    def get_root(self):
        return self.tree[-1][0]

    def get_proof(self, leaf_index):
        """Generate a Merkle proof for a leaf node
        Returns a list of (position, hash) tuples, where position is 'left' or 'right'
        """
        proof = []
        current_index = leaf_index

        for level in range(len(self.tree) - 1):  # Exclude root level
            current_level = self.tree[level]
            # Find sibling index (if current is even, sibling is right; if odd, sibling is left)
            is_right = current_index % 2
            sibling_index = current_index - 1 if is_right else current_index + 1

            # If sibling exists, add it to proof
            if sibling_index < len(current_level):
                position = "left" if is_right else "right"
                proof.append((position, current_level[sibling_index]))

            # Move to parent index in next level
            current_index = current_index // 2

        return proof

    def verify_proof(self, leaf, proof):
        """Verify a Merkle proof
        leaf: The leaf node value to verify
        proof: List of (position, hash) tuples
        """
        current_hash = hash(leaf)

        for position, sibling_hash in proof:
            if position == "left":
                combined = sibling_hash + current_hash
            else:
                combined = current_hash + sibling_hash
            current_hash = hash(combined)

        return current_hash == self.get_root()


# Example usage:
leaves = ["A", "B", "C", "D"]
merkle_tree = MerkleTree(leaves)

# Get proof for "B" (index 1)
proof = merkle_tree.get_proof(1)
print("Merkle Proof:", proof)
# proof[0] is A
assert(proof[0] == ("left", merkle_tree.tree[0][0]))
# proof[1] is hash(C|D)
assert(proof[1] == ("right", merkle_tree.tree[1][1]))

# Verify the proof
is_valid = merkle_tree.verify_proof("B", proof)
print("Proof is valid:", is_valid)

Merkle Proof: [('left', '559aead08264d5795d3909718cdd05abd49572e84fe55590eef31a88a08fdffd'), ('right', '26b5aabe804fe5d533c663dea833e8078188376ce5ca2b5c3371d09ef6b0657b')]
Proof is valid: True


# 2. Generalizing Merkle Trees
## Why Generalize Merkle Trees?
Traditional Merkle Trees assume that all data blocks are of the same size and structure. However, in many applications, data can be more complex and varied. Generalized Merkle Trees extend the concept to handle more diverse data structures.
## What is a Generalized Merkle Tree?
A Generalized Merkle Tree is an advanced version of a Merkle Tree that can accommodate different types of data, such as matrices with different dimensions.

## Example

### Matrices with same dimensions
4 matrices with same dimensions, 8 rows, 1 column.
- Node l0 is the hash of the first row of each matrix.
- Node l1 is the hash of the second row of each matrix.
- ...
- Node l7 is the hash of the last row of each matrix.

![same dimensions](https://hackmd.io/_uploads/BkGW-fFYT.png)

### Matrices with different dimensions
4 matrics:
- 4x3, 4x2, 4x3 -> leaves
- 2x2 -> upper layer

![different dimensions](./mt.png)

# 3. Understanding MMCS (Mixed Matrix Commitment Scheme)
MMCS is a generalization of a vector commitment scheme.
It supports committing to matrices and then opening rows. It is also batch-oriented; one can commit to a batch of matrices at once even if their widths and heights differ.

## How MMCS Works:
MMCS built upon Generalized Merkle Trees. Since Generalized Merkle Trees can handle different types of data, MMCS can handle matrices with different dimensions.

1. commit: build a Generalized Merkle Tree from the matrices.
2. open_batch: prove a node is in the tree <=> prove a row is in the matrix.
3. verify_batch: verify a node is in the tree <=> verify the row is in the matrix.

## Why MMCS?
In STARKs, it requires to commit to multiple polynomials with different degrees: 
- Execution Trace. Multiple columns, each column is a polynomial.
- Quotient Composition Polynomial.
- Polynomials in FRI folding. Fold once, one new polynomial.

- Without MMCS, one polynomial use one merkle tree. 
- With MMCS, we can commit all polynomials in one Generalized Merkle Tree.

Reference: https://hackmd.io/@0xKanekiKen/H1ww-qWKa